In [ ]:
import numpy as np
import imageio.v2 as imageio
import math
import hough_transform as ht
import scipy.ndimage as ndimage
import os
import matplotlib.pyplot as plt
import matplotlib.transforms as trans

os.chdir('C:\\Github\\repository\\image_classification')

#def hough_line(img, angle_step=1, lines_are_white=True, value_threshold=5):
    # Rho and Theta ranges
angle_step = 1
thetas = np.deg2rad(np.arange(-90.0, 90.0, angle_step))
#width, height = img.shape
width = 5#200000
height = 5#200000
diag_len = int(round(math.sqrt(width * width + height * height)))
rhos = np.linspace(-diag_len, diag_len, diag_len * 2)
print('diag_len',diag_len)
# Cache some resuable values
cos_t = np.cos(thetas)
sin_t = np.sin(thetas)
num_thetas = len(thetas)

# Hough accumulator array of theta vs rho
accumulator = np.zeros((2 * diag_len, num_thetas), dtype=np.uint8)
#print('accumulator',accumulator)
# (row, col) indexes to edges
#print('img',img)
#are_edges = img > value_threshold if lines_are_white else img < value_threshold
#print('are_edges',are_edges)
#y_idxs, x_idxs = np.nonzero(are_edges)
x_idxs = np.array([1,2,3])
y_idxs = np.array([1,2,3])
#print('y_idxs',y_idxs)
#print('x_idxs',x_idxs)
# Vote in the hough accumulator
for i in range(len(x_idxs)):
    x = x_idxs[i]
    y = y_idxs[i]

    for t_idx in range(num_thetas):
        # Calculate rho. diag_len is added for a positive index
        rho = diag_len + int(round(x * cos_t[t_idx] + y * sin_t[t_idx]))
        accumulator[rho, t_idx] += 1

neighborhood_size = 20
threshold = 140

data_max = ndimage.maximum_filter(accumulator, neighborhood_size)
maxima = (accumulator == data_max)

data_min = ndimage.minimum_filter(accumulator, neighborhood_size)
diff = ((data_max - data_min) > threshold)
maxima[diff == 0] = 0

labeled, num_objects = ndimage.label(maxima)
slices = ndimage.find_objects(labeled)

x, y = [], []
for dy,dx in slices:
    x_center = (dx.start + dx.stop - 1)/2
    x.append(x_center)
    y_center = (dy.start + dy.stop - 1)/2    
    y.append(y_center)


line_index = 1

for i,j in zip(y, x):

    r = round( (1.0 * i * r_max ) / r_dim,1)
    theta = round( (1.0 * j * theta_max) / theta_dim,1)

    fig, ax = plt.subplots()
    ax.autoscale(False)

    px = []
    py = []
    for i in range(-y_max-neighborhood_size,y_max+neighborhood_size,1):
        px.append( math.cos(-theta) * i - math.sin(-theta) * r ) 
        py.append( math.sin(-theta) * i + math.cos(-theta) * r )

    ax.plot(px,py, linewidth=10)

    plt.savefig("image_line_"+ "%02d" % line_index +".png",bbox_inches='tight')

print('x',x)
print('y',y)
print('len(accumulator)',len(accumulator))
print('accumulator',accumulator)
print('len(maxima)',len(maxima))
print('maxima',maxima)
print('slices',slices)
#print('thetas', thetas)
#print('rhos', rhos)
# imgpath = 'C:\\Github\\repository\\hough_transform-master\\imgs\\binary_crosses.png'
# img = imageio.imread(imgpath)
#ht.show_hough_line(img, accumulator, thetas, rhos, width, height)#, save_path='C:\\Github\\repository\\hough_transform-master\\imgs\\output.png')
fig, ax = plt.subplots(1, 1, figsize=(4, 4))

print('ax',ax)
# ax[1].set_aspect('equal', adjustable='box')
# ax[1].set_title('Hough transform')
# ax[1].set_xlabel('Angles (degrees)')
# ax[1].set_ylabel('Distance (microns)')
# ax[1].axis('image')
plt.figure(figsize=[10,10], dpi=200)
# plt.savefig('C:\\Github\\repository\\hough_transform-master\\imgs\\output.png', bbox_inches=trans.Bbox([[0,1],[0,1]]))
#plt.show()


In [ ]:
import numpy as np
import imageio.v2 as imageio
import math
import hough_transform as ht
import scipy.ndimage as ndimage
import os
import matplotlib.pyplot as plt
import matplotlib.transforms as trans
import cv2

os.chdir('C:\\Github\\repository\\image_classification')
imgpath = 'C:\\Github\\repository\\pentagon.png'
img = imageio.imread(imgpath)
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
plt.imshow(img)
ht.hough_line(img, angle_step=1, lines_are_white=True, value_threshold=5)
# value_threshold=5
# lines_are_white=True
# are_edges = img > value_threshold if lines_are_white else img < value_threshold
# print('len(are_edges)',len(are_edges))
# print('are_edges',are_edges)
# y_idxs, x_idxs = np.nonzero(are_edges)
# print('len(y_idxs)',len(y_idxs))
# print('y_idxs',y_idxs)
# print('len(x_idxs)',len(x_idxs))
# print('x_idxs',x_idxs)

In [ ]:
# Create binary image and call hough_line
image = np.zeros((5,5))
image[1:4, 1:4] = np.eye(3)
plt.imshow(image)
accumulator, thetas, rhos = ht.hough_line(image, lines_are_white=False)
print("len accumulator out",len(accumulator))
print("accumulator out",accumulator)
print("rhos out ",rhos) 
# Easiest peak finding based on max votes
idx = np.argmax(accumulator)
print("accumulator.shape[1] out",accumulator.shape[1]) 
print("idx out",idx)
rho = rhos[idx / accumulator.shape[1]]
theta = thetas[idx % accumulator.shape[1]]
print("rho={0:.2f}, theta={1:.0f}".format(rho, np.rad2deg(theta)))

In [ ]:
image = np.zeros((5,5))
image[1:4, 1:4] = np.eye(3)
plt.imshow(image)

img_shape = image.shape

x_max = img_shape[0]
y_max = img_shape[1]

theta_max = 1.0 * math.pi 
theta_min = 0.0

r_min = 0.0
r_max = math.hypot(x_max, y_max)

r_dim = 200 
theta_dim = 300

hough_space = np.zeros((r_dim,theta_dim))

for x in range(x_max):
    for y in range(y_max):
        if img[x,y] == 255: continue
        for itheta in range(theta_dim):
            theta = 1.0 * itheta * theta_max / theta_dim
            r = x * math.cos(theta) + y * math.sin(theta)
            ir = r_dim * ( 1.0 * r ) / r_max
            hough_space[ir,itheta] = hough_space[ir,itheta] + 1

plt.imshow(hough_space, origin='lower')
plt.xlim(0,theta_dim)
plt.ylim(0,r_dim)

tick_locs = [i for i in range(0,theta_dim,40)]
tick_lbls = [round( (1.0 * i * theta_max) / theta_dim,1) for i in range(0,theta_dim,40)]
plt.xticks(tick_locs, tick_lbls)

tick_locs = [i for i in range(0,r_dim,20)]
tick_lbls = [round( (1.0 * i * r_max ) / r_dim,1) for i in range(0,r_dim,20)]
plt.yticks(tick_locs, tick_lbls)

plt.xlabel(r'Theta')
plt.ylabel(r'r')
plt.title('Hough Space')

plt.imshow(hough_space)
#plt.savefig("hough_space_r_theta.png",bbox_inches='tight')

#plt.close()

In [20]:
import h5py
import numpy as np
# arr = (np.eye(200000)*200000).astype('uint8')
with h5py.File('random.hdf5', 'w') as f:
    dset = f.create_dataset("default", data=(np.eye(200000)*200000).astype('uint8'))
# h5.Group.create_group(grp,1,'Mtrx')
# dset = h5.Group(grp).create_dataset("LargeArray")#, (200000,200000,3),'f')


MemoryError: Unable to allocate 298. GiB for an array with shape (200000, 200000) and data type float64

In [46]:
import tables
# fileh = tb.open_file("objecttree.h5", mode = "w")
# h5f.close()
if h5f.isopen: 
    h5f.close()
h5f = tables.open_file('database_name.h5', 'w')
# grp = tables.Group(tables.Node._v_parent,"Parent_matrix","Matrices")
group = h5f.create_group('/','Matrices')
tables.array(h5f, 'lrg_mtrx', (np.eye(200000)*200000).astype('uint8'))
h5f.close()

MemoryError: Unable to allocate 298. GiB for an array with shape (200000, 200000) and data type float64